# Análisis Semántico Latente
En este cuaderno se realizará la implementación del modelo de Analisis Semántico Latente (LSA) para cada uno de los documentos dentro del corpus.

Esta etapa esta compuesta de las siguientes fases:
1. Cargar datos.
2. Tokenizar por frases el corpus.
3. Aplicar el preprocesamiento al nuevo corpus.
4. Crear diccionarios para relacionar frases e identificadores.
5. Construir la matriz de A del Análisis Semántico Latente.
6. Aplicar la técnica de Análisis Semántico Latente.
7. Generación de resumen

In [3]:
#Importar elementos necesarios de las librerías
import os, shutil, re, pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import PlaintextCorpusReader
from nltk.stem.snowball import SpanishStemmer
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [59]:
#Funciones auxiliares

def tokenize_sentence(path, file_name):
    #Retorna un documento tokenizado por frases
    doc = []
    text = PlaintextCorpusReader(path, file_name)
    paragraphs = text.paras()
    for paragraph in paragraphs:
        for sentence in paragraph:
            low, i = 0,0
            while i < len(sentence):
                token = sentence[i].split('.')
                if len(token)-1:
                    doc.append(sentence[low:i])
                    low=i+1
                    i+=2
                else:
                    i+=1
            if low!=i-1:
                doc.append(sentence[low:i])
    return doc

def preprocess(doc, stopwords, stemmer):
    #Aplica el preprocesamiento establecido
    doc_preprocesed = []
    for sentences in doc:
        sentence = []
        for token in sentences:
            if stemmer.stem(token) not in stopwords:
                sentence.append(stemmer.stem(token))
        if len(sentence) and sentence not in doc_preprocesed:
            doc_preprocesed.append(sentence)
    return doc_preprocesed

def get_dictionaries(doc):
    #Retorna un par de diccionarios que relacionan una frase con un id, y un id con una frase.
    sentence2id, id2sentence = {},{}
    n_sentences = len(doc)
    for i in range(n_sentences):
        sentence = ' '.join(doc[i])
        if sentence not in sentence2id:
            sentence2id[sentence] = i
            id2sentence[i] = sentence
    return sentence2id, id2sentence

def getid2token(token2id):
    #Retorna un diccionario de tokens a id, a partir de un diccionario de id a tokens
    id2token ={}
    for k,v in token2id.items():
        id2token[v] = k
    return id2token

def build_A_Matrix(document, tf_idf, token2id, doc_id):
    #Construye la matriz A que recibe el modelo de LSA como entrada
    data,row_index,col_index = [],[],[]
    tf_idf = tf_idf.toarray()
    n,m = len(document), len(token2id)
    for i in range(n):
        sentence = document[i]
        j = 0
        for token in sentence:
            if token in token2id and tf_idf[doc_id,token2id[token]] != 0:
                if (j==0) or (j>0 and token2id[token] not in col_index[-j:]):
                    value = tf_idf[doc_id,token2id[token]]
                    tf_idf_value = value
                    data.append(tf_idf_value)
                    row_index.append(i)
                    col_index.append(token2id[token])
                    j+=1
    data = np.array(data)
    row_index = np.array(row_index)
    col_index = np.array(col_index)
    A_matrix = csr_matrix((data,(row_index,col_index)),shape=(n, m),dtype=np.float64)
    return A_matrix

def generate_summary(lsa, n_sentences, sentence2id):
    #Genera un resumen con n_sentences frases
    total_sentences = lsa.shape[0]
    assert n_sentences < total_sentences
    columns = ["topic {}".format(i) for i in range(n_sentences)]
    df = pd.DataFrame(lsa,columns=columns)
    df['sentence'] = sentence2id.keys()
    summary = []
    for i in range(n_sentences):
        df = df.sort_values(by='topic {}'.format(i), ascending = False)
        j = 0
        while j < total_sentences:
            sentence = df.iloc[j]['sentence']
            if sentence not in summary:
                summary.append(sentence)
                j=total_sentences
            else:
                j+=1
    return summary

## Fase 1. Cargar datos.
En esta fase se cargarán datos obtenidos durante el preprocesamientoy que utilizaremos para obtener el modelo de LSA.

In [5]:
#Cargar el modelo de tf-idf obtenido en preprocesamiento.
filename = 'tf-idf_model.pkl'
tf_idf = pickle.load(open(filename, 'rb'))

#Cargar la matriz de tf-idf obtenido en preprocesamiento.
filename = 'tf-idf_matrix.pkl'
tf_idf_matrix = pickle.load(open(filename, 'rb'))

#Cargar la matriz de tf-idf obtenido en preprocesamiento.
filename = 'stopwords.pkl'
stopwords = pickle.load(open(filename, 'rb'))

## Fase 2. Tokenizar las frases del corpus.
En esta fase se tokenizará por el corpus por frases.

In [30]:
ruta = "D:/Documents/Documentos Universidad/Noveno/Proyecto de grado/textos"
stemmer = SpanishStemmer()
documento = tokenize_sentence(ruta, 'segurosbolivar-privacidad.txt')
#documento

## Fase 3. Aplicar el preprocesamiento al nuevo corpus.
En esta fase se aplicará el preprocesamiento establecido

In [31]:
documento = preprocess(documento, stopwords, stemmer)
#documento

## Fase 4. Crear diccionarios para relacionar frases e identificadores.
Con el fin de obtener un resumen luego de aplicar la técnica de LSA, se necesita relacionar una frase con un identificador único. Esta fase cumple con dicho objetivo.

In [32]:
frase2id, id2frase = get_dictionaries(documento)
#id2frase

## Fase 5. Construir la matriz de A del Análisis Semántico Latente.
Hasta el momento, ya tenemos una lista de frases preprocesadas, ahora lo siguiente que tenemos que hacer es construir una matriz A de NxM, con N frases y M tokens. Para esto nos apoyaremos en el TF-IDF del preprocesamiento.

In [33]:
token2id = tf_idf.vocabulary_
matriz_A = build_A_Matrix(documento, tf_idf_matrix, token2id, 2)
#matriz_A.toarray()

## Fase 6. Aplicar la técnica de Análisis Semántico Latente.
Una vez construida la matriz A, se utilizará la técnica de LSA para obtener una matriz de MxK, con M frases y K temas.

In [64]:
k = 3

#Instancia un modelo de LSA
lsa_model = TruncatedSVD(n_components=k)

#Aplica el LSA a la matriz tf-idf
lsa = lsa_model.fit_transform(matriz_A)

In [65]:
#Dimensión de la matriz de LSA
lsa.shape

(55, 3)

## Fase 7. Generación del resumen
A partir del modelo de LSA obtenido en la fase anterior, se genera un resúmen escogiendo una frase por cada tema obtenido dentro del modelo de LSA. 

In [66]:
resumen = generate_summary(lsa,k,frase2id)
resumen

[', pon disposicion present avis privac busc inform titul dat personal acerc existent polit tratamient inform sid adopt compañ , form acced caracterist tratamient pretend dar dat',
 'conoc , actualiz rectific dat personal frent respons tratamient encarg tratamient',
 'compañ identific dat administr , asi activ desarroll dich dat , particul recepcion , conserv , disposicion fin propi contrat desarroll activ complementari refer promocion mercade product servici , asi ofrec compañ hac part grup boliv , pertenec asi indic polit tratamient dispon consult pagin web']